In [1]:
! pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "ollama2"

# import dotenv

# dotenv.load_dotenv()

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "ls__9f22c18acdfc480686c135fa4113db26"


In [4]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
! pip install -qU langchain-openai

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="llama2",base_url="http://pt001:11434/v1")

In [7]:
from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(base_url="http://pt001:11434")
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(base_url="http://127.0.0.1:11434/v1", model="llama2", api_key="ollama"))
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

In [ ]:
# cleanup
vectorstore.delete_collection()

## Indexing:Split

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
len(all_splits[0].page_content)

In [ ]:
all_splits[10].metadata

# Indexing: Store

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings(base_url="http://localhost:11434", model="llama2"))

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

In [ ]:
len(retrieved_docs)

In [ ]:
print(retrieved_docs[0].page_content)

## 5. Retrieval and Generation: Generate

In [ ]:
! pip install -qU langchain-openai

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = "ollama2" 

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="llama2",base_url="http://pt001:11434/v1")

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

In [ ]:
print(example_messages[0].content)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")